In [2]:
import pandas as pd
import numpy as np

In [3]:
raw = pd.read_csv('review_data_subset.csv')
data = raw[['verified', 'reviewText']]
True_False = {False: 0, True: 1}
data.replace({"verified": True_False}, inplace = True)
corpus = data['reviewText'].values
labels = data['verified'].values

C:\Users\wang-\ML\my_env\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\wang-\ML\my_env\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [6]:
X = pd.read_csv('vectorized_corpus.csv', index_col = 0)
X = X.values

In [8]:
df = pd.DataFrame(list(zip(labels, X)), columns = ['verified', 'vectorized_corpus'])

In [36]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df)

X_train = train.iloc[:, 1]
X_test = test.iloc[:, 1]
y_train = train.iloc[:, 0]
y_test = test.iloc[:, 0]

In [63]:
# array of array incurs error with logistic regression. 
# need to apply the following transformation to X_test and X_train and make them multidimensional arrays.
def series_to_array(series):
    L = []
    for i in series:
        l = []
        for a in i: 
            l.append(a)
        L.append(l)
    return np.array(L)

In [64]:
X_train_array = series_to_array(X_train)
X_test_array = series_to_array(X_test)

In [66]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter = 1000)
lr.fit(X_train_array, y_train)

LogisticRegression(max_iter=1000)

In [69]:
y_pred = lr.predict(X_train_array)

np.mean(y_pred == y_train)

0.7791466666666667

In [70]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train, y_pred)

array([[ 7197, 12816],
       [ 3748, 51239]], dtype=int64)

In [71]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train, y_pred)

0.6457273419884692

In [78]:
lr.fit(X_train_array, y_train)

LogisticRegression(max_iter=1000)

In [79]:
y_pred = lr.predict(X_test_array)

np.mean(y_pred == y_test)

0.775

### Cross validation

In [82]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(lr, series_to_array(df['vectorized_corpus']), df['verified'], cv=10,return_train_score=False)

In [83]:
cv_results

{'fit_time': array([13.67145824, 11.56526184, 13.90373492, 13.67203355, 13.13864994,
        13.1452899 , 11.66384983, 12.11314011, 12.64546061,  9.1454432 ]),
 'score_time': array([0.00897551, 0.00797796, 0.00797772, 0.00797844, 0.01595926,
        0.01695514, 0.00698233, 0.0079782 , 0.00797772, 0.00797915]),
 'test_score': array([0.7751, 0.78  , 0.7812, 0.7784, 0.777 , 0.779 , 0.7779, 0.7703,
        0.7755, 0.7802])}